In [9]:
#!pip install dotenv

In [15]:
import os, fitz

In [16]:
print(fitz.__doc__)

PyMuPDF 1.24.11: Python bindings for the MuPDF 1.24.10 library (rebased implementation).
Python 3.8 running on win32 (64-bit).



# 1. Data Source

In [28]:
from dotenv import load_dotenv

load_dotenv()
load_dotenv(override=True)

FOLDER_PATH = os.getenv("FIN_DOC_DIR") # Data folder 
#print("Data folder:", FOLDER_PATH)

In [29]:
def txt_from_pdf(folder_path):
    txt = {}
    
    for file in os.listdir(folder_path):
        if file.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            docu = fitz.open(file_path)
            texts = "\n".join([page.get_text() for page in docu])
            txt[file] = texts
    return txt

In [30]:
all_txt = txt_from_pdf(FOLDER_PATH)